# Titanic Survival Prediction
Project ini bertujuan untuk memprediksi siapa yang selamat (**Survived = 1**) dan siapa yang tidak (**Survived = 0**) dari penumpang Titanic menggunakan teknik Machine Learning.

Dataset yang digunakan berasal dari kompetisi Kaggle: *Titanic - Machine Learning from Disaster*.

---

# Workflow / Steps

## 1. Import Data & Cek Struktur
Pada tahap ini dilakukan:
- Load dataset (`train.csv` dan `test.csv`)
- Cek jumlah baris dan kolom
- Cek missing values
- Cek tipe data
- Cek distribusi target (`Survived`)

---

## 2. Data Cleaning
Membersihkan data agar siap diproses:
- Menangani missing values
- Menghapus kolom yang tidak relevan (jika perlu)
- Konversi tipe data kategori menjadi numerik

---

## 3. Feature Engineering
Membuat fitur baru untuk meningkatkan akurasi model:
- Ekstraksi **Title** dari kolom `Name`
- Membuat fitur baru:
  - `FamilySize = SibSp + Parch + 1`
  - `IsAlone` (1 = sendirian, 0 = ada keluarga)
  - `AgeGroup` (binning umur)
  - `FareGroup` (binning harga tiket)
- One-hot encoding untuk fitur kategori:
  - `Sex`
  - `Embarked`
  - `Pclass`

---

## 4. Exploratory Data Analysis (EDA)
Melakukan analisis data menggunakan visualisasi:
- Korelasi antara fitur dengan survival rate
- Gender vs Survival
- Class vs Survival
- Distribusi umur
- Distribusi harga tiket
- Heatmap korelasi

---

## 5. Modeling
Menggunakan beberapa algoritma untuk membandingkan performa:

### - Logistic Regression
Model baseline untuk klasifikasi biner.

### - Decision Tree
Model non-linear yang mudah diinterpretasi.

### - XGBoost
Model boosting dengan akurasi tinggi dan sering unggul di leaderboard Kaggle.

---

## 6. Evaluation
Melakukan evaluasi model:
- Train-test split / cross-validation
- Metrik evaluasi:
  - Accuracy
  - Precision
  - Recall
  - F1-Score
  - Confusion Matrix
- Memilih model terbaik untuk memprediksi dataset test Kaggle.

---

## Submission Format (Kaggle)
CSV final format:

| PassengerId | Survived |
|-------------|----------|
| 892         | 0        |
| 893         | 1        |
| 894         | 0        |

